In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

In [4]:
import json
from tqdm import tqdm

In [ ]:
detailed_output_path = "./data/enriched_requirements_detailed.json"

In [7]:
# 프롬프트 생성 함수 (detailed_description 포함)
def generate_prompt(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항을 분석하여 중요도를 판단하는 전문가입니다.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 다음 항목을 평가하세요:

**중요도**: 해당 요구사항이 시스템 전체의 기능, 보안, 안정성에 얼마나 큰 영향을 주는지 판단해 '상', '중', '하' 중 하나로 평가하세요.
- 상 (C, Critical) : 필수적인 사항. 본 사항에 대한 구현 실패는 곧 시스템 전체의 실패를 의미함. 모든 필수 사항들은 일정 내에 반드시 구현되어야 함.
- 중 (I, Important): 시스템 기능적 효율적 측면에서 중요한 사항들. 본 사항에 대한 구현 실패는 revenue나 customer satisfaction에 영향을 미치지만 실제 일정 상에는 영향을 미치지 않음.
- 하 (U, Useful)  : 유용하나 revenue나 customer satisfaction에 심각한 영향을 미치지 않는 사항

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

중요도: <상|중|하>  
"""


In [ ]:
# GPT 응답 처리 함수
def get_importance_and_difficulty(description, detailed_description, module):
    prompt = generate_prompt(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )
        content = response.choices[0].message.content
        importance = next((line.split(":")[1].strip() for line in content.splitlines() if "중요도" in line), "중")
        difficulty = next((line.split(":")[1].strip() for line in content.splitlines() if "난이도" in line), "중")
        return importance, difficulty
    except Exception as e:
        print("Error:", e)
        return "Error", "Error"

In [ ]:

with open(detailed_output_path, "r", encoding="utf-8") as f:
    requirements = json.load(f)

# 평가 수행
for req in tqdm(requirements):
    desc = req.get("description", "")
    detailed_desc = req.get("detailed_description", "")
    module = req.get("responsible_module", "")
    importance, difficulty = get_importance_and_difficulty(desc, detailed_desc, module)
    req["importance"] = importance

importance_output_path = "./data/enriched_requirements_importance.json"

with open(importance_output_path, "w", encoding="utf-8") as f:
    json.dump(requirements, f, ensure_ascii=False, indent=4)

print("📊 평가 완료: enriched_requirements_importance.json 저장됨.")
